# 5.3 - Web Scraping (bs4)


![scraping](images/scraping.png)


Web scraping o raspado web, es una técnica utilizada mediante programas de software para extraer información de sitios web. Usualmente, estos programas simulan la navegación de un humano en la web ya sea utilizando el protocolo HTTP manualmente, o incrustando un navegador en una aplicación.

El web scraping está muy relacionado con la indexación de la web, la cual indexa la información de la web utilizando un robot y es una técnica universal adoptada por la mayoría de los motores de búsqueda. Sin embargo, el web scraping se enfoca más en la transformación de datos sin estructura en la web, como el formato HTML, en datos estructurados que pueden ser almacenados y analizados en una base de datos central, en una hoja de cálculo o en alguna otra fuente de almacenamiento. Alguno de los usos del web scraping son la comparación de precios en tiendas, la monitorización de datos relacionados con el clima de cierta región, la detección de cambios en sitios webs y la integración de datos en sitios webs. 

En los últimos años el web scraping se ha convertido en una técnica muy utilizada dentro del sector del posicionamiento web gracias a su capacidad de generar grandes cantidades de datos para crear contenidos de calidad.

Podríamos pensar que el web scraping es nuestro último recurso a falta de una API o un feed RSS. A falta de una fuente de datos, siempre podemos extraer aquello que sale por pantalla.

### Extracción desde el HTML

Para scrapear una página web, en primer lugar debemos conocer las estructura que tiene el HTML. Veamos la estructura básica.

El HTML consiste en contenido `<etiquetado>`, es como si fueran cajas de contenido, organizado de manera jerárquica:

```
<html>
    <head>
        <title>Titulo de la pagina</title>
    </head>
    <body>
        <h1>Cabecera</h1>
        <p>Parrafo</p>
    </body>
</html>
```

$$$$

Las etiquetas el HTML se pueden clasificar en varios grupos, dependiendo del tipo de contenido que posea. Estos son algunos ejemplos:

+ cabecera: `<h1>`, `<h2>`, `<h3>`, `<hgroup>`...
+ texto: `<b>`, `<p>`...
+ embebido: `<audio>`, `<img>`, `<video>`...
+ tabular: `<table>`, `<tr>`, `<td>`, `<tbody>`...
+ secciones: `<header>`, `<section>`, `<article>`...
+ metadata: `<meta>`, `<title>`, `<script>`...

$$$$

Las etiquetas pueden tener atributos. Por ejemplo:
 
`<div class="text-monospace" id="name_132", href="www.example.com"> Contenido de la pagina </div>` 

Esta etiqueta `div` tiene los siguientes atributos:

+ class: atributo con valor "text-monospace". La clase no es única en la página.
+ id: atributo con valor "name_132". El id de una etiqueta la identifica de manera unívoca.
+ href: atributo con valor "www.example.com". El href suele contener el link a otra parte de la página.

Siguiendo con la analogía de las cajas, si una etiqueta de HTML es una caja, los atributos serían las pegatinas pegadas en la tapa de la caja.

Conociendo cual es el contenido que queremos extraer, debemos encontrar las etiquetas dentro del HTML de la página web.

Usaremos la herramienta **[BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)**.

In [1]:
!pip install beautifulsoup4

In [2]:
import requests as req

from bs4 import BeautifulSoup as bs

### Ejemplos Wikipedia

**[Países europeos según esperanza de vida](https://en.wikipedia.org/wiki/List_of_European_countries_by_life_expectancy)**

In [3]:
url='https://en.wikipedia.org/wiki/List_of_European_countries_by_life_expectancy'

In [24]:
# usamos requests para extraer el HTML

html=req.get(url).content # o .text

html[:1000]

b'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of European countries by life expectancy - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"91d84c4c-b45d-412c-8702-5972647193b2","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_European_countries_by_life_expectancy","wgTitle":"List of European countries by life expectancy","wgCurRevisionId":1049636953,"wgRevisionId":1049636953,"wgArticleId":22175559,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is differen

In [25]:
# bs4 para el manejo del html

soup=bs(html, 'html.parser')

type(soup)

bs4.BeautifulSoup

In [26]:
# unica tabla en la pagina

tabla=soup.find('table')  # find saca solo el primero, devuelve elemento

type(tabla)

bs4.element.Tag

In [27]:
# extraccion de las filas de la tabla

filas=tabla.find_all('tr')   # find_all saca todos, devuelve una lista

filas=[f.text.strip().split('\n') for f in filas]

filas[:5]

[['Rank',
  '',
  'Country',
  'Life expectancy[1]',
  '',
  'Influenza vaccination rate, people aged 65 and over, 2016 (%)[2]'],
 ['1', '', '\xa0Monaco[3]', '', '89.4'],
 ['2', '', '\xa0San Marino[4]', '', '83.4'],
 ['3', '', '\xa0\xa0Switzerland', '83.0'],
 ['4', '', '\xa0Spain', '82.8', '', '56%']]

In [28]:
# minima limpieza, quitar strings vacias

final=[]

for fila in filas:
    
    tmp=[]
    
    for palabra in fila:
        
        if palabra!='':
            
            tmp.append(palabra)
            
    final.append(tmp)
    
final[:5]

[['Rank',
  'Country',
  'Life expectancy[1]',
  'Influenza vaccination rate, people aged 65 and over, 2016 (%)[2]'],
 ['1', '\xa0Monaco[3]', '89.4'],
 ['2', '\xa0San Marino[4]', '83.4'],
 ['3', '\xa0\xa0Switzerland', '83.0'],
 ['4', '\xa0Spain', '82.8', '56%']]

In [29]:
import pandas as pd

col_names=final[0]

data=final[1:]

df=pd.DataFrame(data, columns=col_names)

df.head(10)

,Rank,Country,Life expectancy[1],"Influenza vaccination rate, people aged 65 and over, 2016 (%)[2]"
0,1,Monaco[3],89.4,None
1,2,San Marino[4],83.4,None
2,3,Switzerland,83.0,None
3,4,Spain,82.8,56%
4,5,Liechtenstein,82.7,28%
5,6,Italy,82.7,50%
6,7,Norway,82.5,38%
7,8,Iceland,82.5,47%
8,9,Luxembourg,82.3,38%
9,10,France,82.3,50%


$$$$

**[Medallero Barcelona'92](https://es.wikipedia.org/wiki/Juegos_Ol%C3%ADmpicos_de_Barcelona_1992)**

$$$$

**[Videoconsolas](https://es.wikipedia.org/wiki/Videoconsola)**

### Ejemplo geolocalización por IP

https://tools.keycdn.com/geo

**¿Dónde estoy?**

**Búsqueda según IP**

https://tools.keycdn.com/geo?host=137.255.90.7

### Ejemplo terremotos (EMSC)

https://www.emsc-csem.org/Earthquake/

### Ejemplo LinkedIn